# Part 7.6: Bunching 方法 (Bunching Design)

## 学习目标

1. 理解 Bunching 的核心思想和适用场景
2. 掌握弹性估计的理论框架
3. 学习缺失质量 (Missing Mass) 的计算
4. 实现从零 Bunching 分析算法
5. 应用于税收和政策阈值分析

---

## 业务场景：税收政策的行为响应

想象你是某国税务部门的经济学家。政府规定：
- 年收入低于 50,000 美元的税率是 20%
- 年收入超过 50,000 美元的税率是 35%

**观察到的现象**：
- 在 50,000 美元附近，收入分布出现明显的「堆积」(Bunching)
- 很多人报告收入恰好在 49,999 美元
- 50,000 美元之上的收入明显减少

**核心问题**：
1. 纳税人对税率变化的行为响应有多大？
2. 收入弹性是多少？
3. 如何量化「避税行为」的程度？

这就是 **Bunching 方法** 要解决的问题！

---

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from scipy.optimize import minimize
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# 颜色配置
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0'
}

print("✅ 环境准备完成！")

## Part 1: 核心概念

### 什么是 Bunching？

**定义**：当存在政策阈值或税率变化点时，个体会调整行为使其聚集在阈值附近，导致分布出现「堆积」现象。

### 典型应用场景

1. **税收政策**
   - 个人所得税税率跳跃
   - 企业税优惠阈值
   - 增值税起征点

2. **劳动经济学**
   - 失业救济金领取条件
   - 退休金计算阈值
   - 工时规定

3. **教育政策**
   - 奖学金分数线
   - 入学资格阈值

### 核心思想

```
反事实分布（无政策时的平滑分布）
      |
      |     实际分布
      |      /\     
      |     /  \    Bunching（堆积）
      |    /    \   
      |___/______\___
      |   阈值点
      |
      |---缺失质量（向左移动的人）
```

**关键量**：
- **Bunching (B)**: 阈值处的超额密度
- **Missing Mass**: 阈值右侧缺失的密度
- **Elasticity (ε)**: 行为响应弹性

---

## Part 2: 理论框架

### 优化问题

个体 i 最大化效用：
$$\max_z U(c, z) = c - \frac{n}{1+\frac{1}{\varepsilon}}z^{1+\frac{1}{\varepsilon}}$$

其中：
- $c$: 消费 (收入减税后)
- $z$: 报告收入
- $\varepsilon$: 应税收入弹性
- $n$: 个体能力异质性参数

预算约束：
$$c = z - T(z)$$

其中 $T(z)$ 是税收函数。

### 税率跳跃

假设：
- $z < z^*$ 时，税率 $t_1$
- $z \geq z^*$ 时，税率 $t_2 > t_1$

在 $z^*$ 处，边际税率从 $t_1$ 跳跃到 $t_2$。

### 反应区间

能力在 $[n^*, n^* + \Delta n^*]$ 的个体会 bunch 到 $z^*$。

**关键关系**：
$$B = \int_{z^*}^{z^* + \Delta z^*} h_0(z) dz$$

其中：
- $h_0(z)$: 反事实密度函数（无税率跳跃时）
- $\Delta z^*$: 反应区间

### 弹性估计公式

**Saez (2010) 公式**：
$$\varepsilon = \frac{B / h_0(z^*)}{\Delta z^* / z^*} \cdot \frac{z^*}{\Delta t / (1 - t_1)}$$

简化为：
$$\varepsilon \approx \frac{B}{h_0(z^*) \cdot \Delta z^*} \cdot \frac{z^*}{\ln(1-t_2) - \ln(1-t_1)}$$

---

## Part 3: 数据生成

In [ ]:
def generate_bunching_data(n=10000, kink_point=50, t1=0.2, t2=0.35, elasticity=0.5, seed=42):
    """
    生成 Bunching 数据
    
    参数:
        n: 样本量
        kink_point: 税率跳跃点 (千美元)
        t1: 低税率
        t2: 高税率
        elasticity: 真实弹性
        seed: 随机种子
    """
    np.random.seed(seed)
    
    # 能力分布（对数正态）
    ability = np.random.lognormal(mean=3.9, sigma=0.4, size=n)
    
    # 反事实收入（无税率跳跃）
    z_counterfactual = ability * 10
    
    # 计算税后收入差异
    def net_income(z, t):
        return z * (1 - t)
    
    # 实际收入（考虑 bunching 行为）
    z_actual = z_counterfactual.copy()
    
    # 计算反应区间
    delta_t = t2 - t1
    log_ratio = np.log(1 - t1) - np.log(1 - t2)
    
    # 简化的 bunching 行为
    # 如果个体在 kink_point 附近，且跳过 kink 会损失效用，则 bunch
    for i in range(n):
        z_cf = z_counterfactual[i]
        
        # 在 kink_point 之上的人
        if z_cf > kink_point:
            # 计算在 kink_point 的效用
            u_kink = net_income(kink_point, t1)
            # 计算在反事实点的效用（近似）
            u_cf = net_income(z_cf, t2) - 0.5 * elasticity * (z_cf - kink_point)**2 / kink_point
            
            # 如果 bunch 更优，则移动到 kink
            if u_kink > u_cf and z_cf < kink_point * 1.3:  # 反应区间限制
                z_actual[i] = kink_point + np.random.normal(0, 0.5)  # 加噪声
    
    # 添加测量误差
    z_actual += np.random.normal(0, 1, n)
    z_actual = np.maximum(z_actual, 10)  # 最低收入
    
    return {
        'income': z_actual,
        'income_cf': z_counterfactual,
        'kink_point': kink_point,
        't1': t1,
        't2': t2,
        'true_elasticity': elasticity
    }

# 生成数据
data = generate_bunching_data(n=10000, kink_point=50, t1=0.2, t2=0.35, elasticity=0.5)
income = data['income']
kink = data['kink_point']

print(f"数据维度: n={len(income)}")
print(f"税率跳跃点: ${kink}k")
print(f"低税率: {data['t1']:.0%}")
print(f"高税率: {data['t2']:.0%}")
print(f"真实弹性: {data['true_elasticity']:.2f}")
print(f"\n收入分布:")
print(f"  均值: ${income.mean():.1f}k")
print(f"  中位数: ${np.median(income):.1f}k")

## Part 4: 可视化 Bunching

In [ ]:
# 绘制收入分布直方图
bins = np.arange(20, 80, 1)
hist, bin_edges = np.histogram(income, bins=bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

fig = go.Figure()

# 直方图
fig.add_trace(go.Bar(
    x=bin_centers,
    y=hist,
    name='实际分布',
    marker_color=COLORS['primary'],
    opacity=0.7
))

# 标记 kink point
fig.add_vline(
    x=kink,
    line_dash="dash",
    line_color=COLORS['danger'],
    annotation_text=f"税率跳跃点 (${kink}k)",
    annotation_position="top"
)

# 标记 bunching 区域
bunching_idx = np.abs(bin_centers - kink) < 2
fig.add_trace(go.Bar(
    x=bin_centers[bunching_idx],
    y=hist[bunching_idx],
    name='Bunching 区域',
    marker_color=COLORS['warning'],
    opacity=0.8
))

fig.update_layout(
    title='收入分布：明显的 Bunching 现象',
    xaxis_title='年收入 (千美元)',
    yaxis_title='频数',
    template='plotly_white',
    height=500,
    showlegend=True
)

fig.show()

print("\n📊 观察:")
print(f"1. 在 ${kink}k 附近出现明显的堆积")
print(f"2. ${kink}k 之上的密度明显下降")
print(f"3. 这表明纳税人在响应税率变化")

## Part 5: 估计反事实分布

### 方法：多项式拟合

**步骤**：
1. 排除 bunching 区间 $[z^* - \delta_L, z^* + \delta_U]$
2. 用剩余数据拟合多项式 $h_0(z) = \sum_{p=0}^P \beta_p z^p$
3. 预测 bunching 区间的反事实密度

**多项式阶数选择**：
- 通常 P = 5 到 9
- 使用 AIC/BIC 或交叉验证

In [ ]:
class BunchingEstimator:
    """
    Bunching 估计器
    实现 Chetty et al. (2011) 的方法
    """
    
    def __init__(self, bin_width=1, poly_degree=7, exclude_width=3):
        """
        参数:
            bin_width: 分箱宽度
            poly_degree: 多项式阶数
            exclude_width: 排除区间的宽度（左右各多少）
        """
        self.bin_width = bin_width
        self.poly_degree = poly_degree
        self.exclude_width = exclude_width
        
    def fit(self, income, kink_point, income_range=None):
        """
        拟合反事实分布
        """
        self.kink_point = kink_point
        
        # 确定分析区间
        if income_range is None:
            income_range = (kink_point - 20, kink_point + 20)
        
        # 创建分箱
        bins = np.arange(income_range[0], income_range[1], self.bin_width)
        hist, bin_edges = np.histogram(income, bins=bins)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        
        # 排除 bunching 区间
        exclude_mask = (
            (bin_centers >= kink_point - self.exclude_width) & 
            (bin_centers <= kink_point + self.exclude_width)
        )
        
        # 拟合多项式（使用非 bunching 区间）
        X_train = bin_centers[~exclude_mask].reshape(-1, 1)
        y_train = hist[~exclude_mask]
        
        poly = PolynomialFeatures(degree=self.poly_degree)
        X_train_poly = poly.fit_transform(X_train)
        
        self.model = LinearRegression()
        self.model.fit(X_train_poly, y_train)
        
        # 预测整个区间的反事实密度
        X_all_poly = poly.transform(bin_centers.reshape(-1, 1))
        h0_pred = self.model.predict(X_all_poly)
        h0_pred = np.maximum(h0_pred, 0)  # 确保非负
        
        # 保存结果
        self.bin_centers = bin_centers
        self.hist_actual = hist
        self.hist_counterfactual = h0_pred
        self.exclude_mask = exclude_mask
        self.poly = poly
        
        return self
    
    def estimate_bunching(self):
        """
        估计 bunching mass
        """
        # Bunching: 实际密度 - 反事实密度（在 bunching 区间内）
        bunching_region = self.exclude_mask
        B = np.sum(self.hist_actual[bunching_region] - self.hist_counterfactual[bunching_region])
        
        # 找到 missing mass 区间（kink 右侧）
        right_mask = (self.bin_centers > self.kink_point + self.exclude_width)
        missing_mass_diff = self.hist_counterfactual[right_mask] - self.hist_actual[right_mask]
        missing_mass_diff = np.maximum(missing_mass_diff, 0)  # 只计算正的部分
        
        # 累计 missing mass 直到与 B 相等
        cumsum_missing = np.cumsum(missing_mass_diff)
        
        # 找到反应区间的右端点
        if len(cumsum_missing) > 0 and cumsum_missing[-1] > 0:
            # 找到累计值最接近 B 的位置
            idx = np.argmin(np.abs(cumsum_missing - B))
            z_upper = self.bin_centers[right_mask][idx]
        else:
            z_upper = self.kink_point + 5  # 默认值
        
        delta_z = z_upper - self.kink_point
        
        return {
            'bunching_mass': B,
            'z_upper': z_upper,
            'delta_z': delta_z
        }
    
    def estimate_elasticity(self, t1, t2):
        """
        估计弹性
        """
        results = self.estimate_bunching()
        B = results['bunching_mass']
        delta_z = results['delta_z']
        
        # h0 在 kink point 的值
        kink_idx = np.argmin(np.abs(self.bin_centers - self.kink_point))
        h0_kink = self.hist_counterfactual[kink_idx]
        
        # 弹性公式
        if h0_kink > 0 and delta_z > 0:
            log_ratio = np.log(1 - t1) - np.log(1 - t2)
            elasticity = (B / h0_kink) / (delta_z / self.kink_point) * (self.kink_point / log_ratio)
        else:
            elasticity = np.nan
        
        return {
            **results,
            'h0_kink': h0_kink,
            'elasticity': elasticity
        }
    
    def plot(self):
        """
        可视化拟合结果
        """
        fig = go.Figure()
        
        # 实际分布
        fig.add_trace(go.Bar(
            x=self.bin_centers,
            y=self.hist_actual,
            name='实际分布',
            marker_color=COLORS['primary'],
            opacity=0.6
        ))
        
        # 反事实分布
        fig.add_trace(go.Scatter(
            x=self.bin_centers,
            y=self.hist_counterfactual,
            name='反事实分布',
            line=dict(color=COLORS['danger'], width=3, dash='dash')
        ))
        
        # 标记 kink point
        fig.add_vline(
            x=self.kink_point,
            line_dash="dot",
            line_color='black',
            annotation_text="Kink Point"
        )
        
        # 标记排除区间
        fig.add_vrect(
            x0=self.kink_point - self.exclude_width,
            x1=self.kink_point + self.exclude_width,
            fillcolor=COLORS['warning'],
            opacity=0.2,
            annotation_text="排除区间",
            annotation_position="top left"
        )
        
        fig.update_layout(
            title='Bunching 分析：实际分布 vs 反事实分布',
            xaxis_title='收入 (千美元)',
            yaxis_title='频数',
            template='plotly_white',
            height=500
        )
        
        return fig

# 应用 Bunching 估计
bunching = BunchingEstimator(bin_width=1, poly_degree=7, exclude_width=3)
bunching.fit(income, kink_point=kink)

# 可视化
fig = bunching.plot()
fig.show()

print("\n✅ 反事实分布估计完成！")
print("虚线表示没有税率跳跃时的收入分布")

## Part 6: 估计弹性

In [ ]:
# 估计弹性
results = bunching.estimate_elasticity(t1=data['t1'], t2=data['t2'])

print("Bunching 分析结果")
print("="*60)
print(f"Bunching Mass (B): {results['bunching_mass']:.1f}")
print(f"反应区间上界: ${results['z_upper']:.1f}k")
print(f"反应区间宽度 (Δz): ${results['delta_z']:.1f}k")
print(f"Kink 处的反事实密度: {results['h0_kink']:.1f}")
print(f"\n估计的弹性 (ε): {results['elasticity']:.3f}")
print(f"真实弹性: {data['true_elasticity']:.3f}")
print(f"估计误差: {abs(results['elasticity'] - data['true_elasticity']):.3f}")

## Part 7: 敏感性分析

### 多项式阶数的影响

In [ ]:
# 测试不同的多项式阶数
poly_degrees = range(4, 12)
elasticities = []
bunching_masses = []

for degree in poly_degrees:
    b = BunchingEstimator(poly_degree=degree, exclude_width=3)
    b.fit(income, kink)
    res = b.estimate_elasticity(t1=data['t1'], t2=data['t2'])
    elasticities.append(res['elasticity'])
    bunching_masses.append(res['bunching_mass'])

# 可视化
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('弹性估计', 'Bunching Mass')
)

# 弹性
fig.add_trace(
    go.Scatter(
        x=list(poly_degrees),
        y=elasticities,
        mode='lines+markers',
        name='估计值',
        line=dict(color=COLORS['primary'])
    ),
    row=1, col=1
)

fig.add_hline(
    y=data['true_elasticity'],
    line_dash="dash",
    line_color=COLORS['danger'],
    annotation_text="真实值",
    row=1, col=1
)

# Bunching Mass
fig.add_trace(
    go.Scatter(
        x=list(poly_degrees),
        y=bunching_masses,
        mode='lines+markers',
        name='Bunching Mass',
        line=dict(color=COLORS['success']),
        showlegend=False
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="多项式阶数", row=1, col=1)
fig.update_xaxes(title_text="多项式阶数", row=1, col=2)
fig.update_yaxes(title_text="弹性", row=1, col=1)
fig.update_yaxes(title_text="Bunching Mass", row=1, col=2)

fig.update_layout(
    title='敏感性分析：多项式阶数的影响',
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n📊 观察:")
print(f"1. 多项式阶数在 6-8 时估计较稳定")
print(f"2. 过低的阶数会欠拟合，过高的阶数会过拟合")
print(f"3. 推荐使用 7 阶多项式")

### 排除区间的影响

In [ ]:
# 测试不同的排除区间
exclude_widths = [1, 2, 3, 4, 5, 6]
elasticities_exclude = []

for width in exclude_widths:
    b = BunchingEstimator(poly_degree=7, exclude_width=width)
    b.fit(income, kink)
    res = b.estimate_elasticity(t1=data['t1'], t2=data['t2'])
    elasticities_exclude.append(res['elasticity'])

# 可视化
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=exclude_widths,
    y=elasticities_exclude,
    mode='lines+markers',
    name='估计值',
    line=dict(color=COLORS['primary'], width=3),
    marker=dict(size=10)
))

fig.add_hline(
    y=data['true_elasticity'],
    line_dash="dash",
    line_color=COLORS['danger'],
    annotation_text="真实值"
)

fig.update_layout(
    title='敏感性分析：排除区间宽度的影响',
    xaxis_title='排除区间宽度（千美元）',
    yaxis_title='估计弹性',
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n📊 观察:")
print(f"1. 排除区间需要足够大以覆盖 bunching 区域")
print(f"2. 但不能太大，否则会丢失有用信息")
print(f"3. 推荐使用 3-4 的宽度")

## Part 8: Bootstrap 置信区间

In [ ]:
def bootstrap_bunching(income, kink, t1, t2, n_bootstrap=500, alpha=0.05):
    """
    Bootstrap 估计弹性的置信区间
    """
    elasticities = []
    n = len(income)
    
    for i in range(n_bootstrap):
        # 重采样
        idx = np.random.choice(n, size=n, replace=True)
        income_boot = income[idx]
        
        # 估计
        try:
            b = BunchingEstimator(poly_degree=7, exclude_width=3)
            b.fit(income_boot, kink)
            res = b.estimate_elasticity(t1, t2)
            
            if not np.isnan(res['elasticity']):
                elasticities.append(res['elasticity'])
        except:
            pass
    
    elasticities = np.array(elasticities)
    
    # 计算置信区间
    ci_lower = np.percentile(elasticities, 100 * alpha / 2)
    ci_upper = np.percentile(elasticities, 100 * (1 - alpha / 2))
    
    return {
        'elasticities': elasticities,
        'mean': np.mean(elasticities),
        'std': np.std(elasticities),
        'ci_lower': ci_lower,
        'ci_upper': ci_upper
    }

# 运行 Bootstrap
print("运行 Bootstrap（这可能需要一些时间）...")
boot_results = bootstrap_bunching(income, kink, data['t1'], data['t2'], n_bootstrap=200)

# 可视化 Bootstrap 分布
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=boot_results['elasticities'],
    nbinsx=30,
    name='Bootstrap 分布',
    marker_color=COLORS['primary']
))

fig.add_vline(
    x=boot_results['mean'],
    line_dash="dash",
    line_color=COLORS['success'],
    annotation_text=f"均值: {boot_results['mean']:.3f}"
)

fig.add_vline(
    x=boot_results['ci_lower'],
    line_dash="dot",
    line_color=COLORS['danger'],
    annotation_text=f"95% CI 下界"
)

fig.add_vline(
    x=boot_results['ci_upper'],
    line_dash="dot",
    line_color=COLORS['danger'],
    annotation_text=f"95% CI 上界"
)

fig.update_layout(
    title='弹性估计的 Bootstrap 分布',
    xaxis_title='弹性',
    yaxis_title='频数',
    template='plotly_white',
    height=400
)

fig.show()

print("\nBootstrap 结果")
print("="*60)
print(f"均值: {boot_results['mean']:.3f}")
print(f"标准误: {boot_results['std']:.3f}")
print(f"95% 置信区间: [{boot_results['ci_lower']:.3f}, {boot_results['ci_upper']:.3f}]")
print(f"\n真实值 {data['true_elasticity']:.3f} 是否在置信区间内: ", end="")
if boot_results['ci_lower'] <= data['true_elasticity'] <= boot_results['ci_upper']:
    print("✅ 是")
else:
    print("❌ 否")

## Part 9: 实际应用案例

### 案例: 个人所得税改革

**背景**：政府计划调整税率阶梯，需要评估纳税人的行为响应。

**问题**：
1. 当前税率跳跃点处的 bunching 有多严重？
2. 应税收入弹性是多少？
3. 如果提高税率，会损失多少税收？

In [ ]:
# 政策模拟
def simulate_tax_reform(elasticity, current_kink, new_tax_rate, income_dist):
    """
    模拟税率改革的影响
    """
    # 当前税收
    current_tax = data['t2']
    
    # 预测新的 bunching 行为
    delta_t = new_tax_rate - current_tax
    
    # 简化的行为响应模型
    # 假设应税收入变化 = -elasticity * (delta_t / (1 - current_tax)) * income
    high_income = income_dist > current_kink
    
    income_new = income_dist.copy()
    income_new[high_income] *= (1 - elasticity * delta_t / (1 - current_tax))
    
    # 计算税收变化
    def calculate_tax(income, kink, t1, t2):
        tax = np.where(
            income <= kink,
            income * t1,
            kink * t1 + (income - kink) * t2
        )
        return tax
    
    tax_current = calculate_tax(income_dist, current_kink, data['t1'], current_tax)
    tax_new_mechanical = calculate_tax(income_dist, current_kink, data['t1'], new_tax_rate)
    tax_new_behavioral = calculate_tax(income_new, current_kink, data['t1'], new_tax_rate)
    
    return {
        'income_new': income_new,
        'tax_current': tax_current.sum(),
        'tax_new_mechanical': tax_new_mechanical.sum(),
        'tax_new_behavioral': tax_new_behavioral.sum(),
        'behavioral_response': tax_new_mechanical.sum() - tax_new_behavioral.sum()
    }

# 模拟提高税率到 40%
reform = simulate_tax_reform(
    elasticity=results['elasticity'],
    current_kink=kink,
    new_tax_rate=0.40,
    income_dist=income
)

print("税率改革影响分析")
print("="*60)
print(f"当前税率: {data['t2']:.0%}")
print(f"新税率: 40%")
print(f"\n税收影响:")
print(f"  当前总税收: ${reform['tax_current']/1000:.1f}M")
print(f"  机械效应（不考虑行为响应）: ${reform['tax_new_mechanical']/1000:.1f}M")
print(f"  考虑行为响应后: ${reform['tax_new_behavioral']/1000:.1f}M")
print(f"  \n行为响应导致的税收损失: ${reform['behavioral_response']/1000:.1f}M")
print(f"  损失占机械效应的比例: {reform['behavioral_response']/(reform['tax_new_mechanical']-reform['tax_current']):.1%}")

## 思考题与练习

### 基础理解

1. **Bunching 方法的核心假设是什么？什么情况下会失效？**

2. **为什么需要估计反事实分布？直接比较 kink 左右两边的密度不行吗？**

3. **弹性估计公式中的各项分别代表什么含义？**

### 深入分析

4. **如果存在多个 kink point（多档税率），如何分析？**

5. **如何判断观察到的 bunching 是真实的行为响应，还是报告误差（misreporting）？**

6. **优化摩擦（optimization frictions）会如何影响 bunching 估计？**

### 面试题

**题目 1**: 某电商平台对月销售额超过 10 万的商家收取 3% 平台费，低于 10 万的收取 1%。观察数据发现，很多商家的月销售额恰好在 9.9 万左右。

- 如何估计商家对平台费的弹性？
- 需要注意哪些潜在问题？
- 如果要优化平台费结构，你会给出什么建议？

**题目 2**: 编码题 - 实现分位数 bunching 检验

```python
def test_bunching_significance(income, kink_point, n_permutations=1000):
    """
    使用置换检验（permutation test）检验 bunching 是否显著
    
    参数:
        income: 收入数据
        kink_point: kink point 位置
        n_permutations: 置换次数
    
    返回:
        p-value
    """
    # TODO: 实现这个函数
    pass
```

---

## 总结

### 核心要点

| 概念 | 定义 | 公式 |
|------|------|------|
| **Bunching (B)** | 阈值处的超额密度 | $B = \int_{z^*-\delta_L}^{z^*+\delta_U} [h_1(z) - h_0(z)] dz$ |
| **Missing Mass** | 阈值右侧缺失的密度 | 应与 B 相等（质量守恒） |
| **反事实分布** | 无政策时的分布 | 通过多项式拟合估计 |
| **弹性 (ε)** | 行为响应的弹性 | $\varepsilon = \frac{B}{h_0(z^*) \cdot \Delta z^*} \cdot \frac{z^*}{\text{log-ratio}}$ |

### 方法步骤

1. **数据可视化**: 绘制分布直方图，识别 bunching
2. **反事实估计**: 排除 bunching 区间，拟合多项式
3. **Bunching 量化**: 计算超额密度和 missing mass
4. **弹性估计**: 使用公式计算弹性
5. **敏感性分析**: 测试多项式阶数和排除区间
6. **置信区间**: Bootstrap 或解析方法

### 优点与局限

**优点**:
- 不需要工具变量或自然实验
- 可以从截面数据估计弹性
- 方法直观，易于可视化

**局限**:
- 需要存在清晰的 kink/notch
- 假设局部线性化
- 难以区分真实响应和报告误差
- 只能估计局部弹性

### 实践建议

1. **仔细选择分箱宽度**: 太宽会平滑掉 bunching，太窄会有噪声
2. **多项式阶数**: 从 5-9 尝试，检查稳定性
3. **排除区间**: 要足够大覆盖 bunching，但不能太大
4. **稳健性检验**: 尝试不同规格，确保结果稳健
5. **可视化**: 始终绘制实际分布 vs 反事实分布

### 延伸阅读

- **经典论文**:
  - Saez (2010): "Do Taxpayers Bunch at Kink Points?"
  - Chetty et al. (2011): "Adjustment Costs, Firm Responses, and Micro vs. Macro Labor Supply Elasticities"
  - Kleven & Waseem (2013): "Using Notches to Uncover Optimization Frictions and Structural Elasticities"

- **Python 工具**:
  - `bunchit` package (如果可用)
  - 自己实现（如本 notebook）

---

**恭喜完成 Bunching 方法的学习！**

你现在可以：
- ✅ 识别和量化 bunching 现象
- ✅ 估计反事实分布
- ✅ 计算行为响应弹性
- ✅ 应用于税收和政策评估
- ✅ 进行稳健性和敏感性分析